In [ ]:
import pandas as pd
from scholarly import scholarly
import time
import csv
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("Computer_Vision.csv")

In [ ]:
df['formatted_authors_name'] = ""
for index, row in df.iterrows():
    lis = (row['authors_name']).split(',')
    # print(lis)
    if lis:
        l = []
        for i in lis:
            i = i.strip()
            i = i.split(' ')
            j = "_".join(i)
            # print(j)
            # j = str(j).replace('[','').replace(']','').replace('\"','')
            j = str(j).replace('[','').replace(']','')
            # print(j)
            l.append((j))
            df.at[index,'formatted_authors_name'] = l

In [ ]:
df = df.drop_duplicates(subset='Author_ID')

In [ ]:
# Input list of author IDs
author_ids = df['authors_id'].tolist()
author_ids = author_ids[0:10]

# Output CSV file
output_file = "CV_authors_info.csv"

# Function to get author info
def get_author_info(author_id):
    try:
        author = scholarly.search_author_id(author_id)
        author_info = scholarly.fill(author)
        return {
            "name": author_info.get("name", ""),
            "affiliation": author_info.get("affiliation", ""),
            "h_index": author_info.get("hindex", 0),
            "citations": author_info.get("citedby", 0),
            'email': author_info.get("email", 0),
        }
    except Exception as e:
        print(f"Error for author {author_id}: {e}")
        return None

# Write data to CSV
with open(output_file, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["Author ID", "Name", "Affiliation", "H-index", "Citations","Email"])
    writer.writeheader()

    for author_id in author_ids:
        author_data = get_author_info(author_id)
        if author_data:
            author_data["Author ID"] = author_id
            writer.writerow(author_data)
            print(f"Scraped: {author_data['name']} ({author_id})")
        time.sleep(5)  # Rate limit (adjust as needed)



In [ ]:
new_df = pd.read_csv("CV_authors_info")
new_df.loc[~new_df['ga_gender'].isin(["male", "female"]), 'ga_gender'] = 'unknown'
new_df.to_csv("CV_authors_infoo.csv")

In [ ]:
gender_counts = df["ga_gender"].value_counts()
gender_percentages = (gender_counts / gender_counts.sum()) * 100

# Plot distribution
plt.figure(figsize=(6, 6))
gender_percentages.plot(kind="pie", autopct="%.1f%%", colors=["#ff9999", "#66b3ff"], startangle=90)
plt.title("Gender Distribution")
plt.ylabel("")  # Removes default ylabel for a cleaner look
plt.show()